In [6]:
import os
import numpy as np
import pandas as pd
from nilearn.image import load_img, mean_img
import matplotlib.pyplot as plt
import seaborn as sns


base_dir = "/Users/kungtzuyun/Lab/ACTI/first_level/"  
n_subs = 20              
n_conds = 130             
subjects = sorted([d for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, d))])
print(f"Find {len(subjects)} participants：{subjects}")

data = []

for sub in subjects:
    sub_dir = os.path.join(base_dir, sub)
    for cond in range(1, n_conds + 1):
        fname = f"con_{cond:04d}.nii"
        fpath = os.path.join(sub_dir, fname)
        if not os.path.exists(fpath):
            print(f"No file: {fpath}")
            continue
        
        con_img = load_img(fpath)
        values = con_img.get_fdata()
        mean_beta = np.nanmean(values[values != 0]) 
        
        data.append({
            "subject": sub,
            "condition": f"cond{cond}",
            "mean_beta": mean_beta
        })

df = pd.DataFrame(data)
df.to_csv("wholebrain_mean_activation.csv", index=False)

plt.figure(figsize=(14, 6))
sns.barplot(data=df, x="condition", y="mean_beta", errorbar="se", capsize=0.1)
plt.title("Whole-brain Mean Activation per Condition")
plt.ylabel("Average Beta (All Voxels)")
plt.xticks(rotation=90)
plt.tight_layout()
plt.savefig("mean_activation_barplot.png", dpi=300)
plt.close()


output_dir = "group_avg_maps"
os.makedirs(output_dir, exist_ok=True)

for cond in range(1, n_conds + 1):
    imgs = []
    for sub in subjects:  
        fpath = os.path.join(base_dir, sub, f"con_{cond:04d}.nii")
        if os.path.exists(fpath):
            imgs.append(load_img(fpath))
    if imgs:
        avg_img = mean_img(imgs)
        avg_img.to_filename(os.path.join(output_dir, f"group_avg_con_{cond:04d}.nii"))

print("Done!")
print("- pics：mean_activation_barplot.png")
print("- Data：wholebrain_mean_activation.csv")
print("- Average pics：group_avg_maps/*.nii")

Find 20 participants：['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'B001', 'B002', 'B003', 'B004', 'B005', 'B006', 'B008', 'B010', 'B011', 'B012']


/var/folders/_w/fjsm684173587hp23d05dbsc0000gn/T/ipykernel_28963/140507007.py:59: FutureWarning: From release 0.13.0 onwards, this function will, by default, copy the header of the input image to the output. Currently, the header is reset to the default Nifti1Header. To suppress this warning and use the new behavior, set `copy_header=True`.
  avg_img = mean_img(imgs)


Done!
- pics：mean_activation_barplot.png
- Data：wholebrain_mean_activation.csv
- Average pics：group_avg_maps/*.nii


In [20]:
import os
from nilearn import plotting, datasets
from nilearn.image import load_img

# === 使用者參數設定 ===
n_conds = 130
input_dir = "group_avg_maps"
output_dir = "stat_map_figures_yellow_red"
os.makedirs(output_dir, exist_ok=True)

mni = datasets.load_mni152_template()

# === 自訂顏色顯示參數 ===
vmin = -13
vmax = 13

# === 逐 contrast 畫圖 ===
for cond in range(1, n_conds + 1):
    img_path = os.path.join(input_dir, f"group_avg_con_{cond:04d}.nii")
    output_file = os.path.join(output_dir, f"cond_{cond:04d}.png")

    if not os.path.exists(img_path):
        print(f"⚠️ 檔案不存在: {img_path}")
        continue

    try:
        display = plotting.plot_stat_map(
            stat_map_img=img_path,
            bg_img=mni,
            threshold=1.38,       # 保留
            display_mode='z',
            cut_coords=[-30, -10, 0, 10, 20, 30, 40],
            title=f"Group Avg - Condition {cond}",
            colorbar=True,
            cmap='cold_hot',
            vmin=vmin,
            vmax=vmax,
            symmetric_cbar=True
        )
        display.savefig(output_file)
        display.close()
        print(f"✅ 成功畫圖: Condition {cond}")
    except Exception as e:
        print(f"❌ Condition {cond} failed with error: {e}")
        print("🔁 改用 threshold=0 再試一次")
        try:
            display = plotting.plot_stat_map(
                stat_map_img=img_path,
                bg_img=mni,
                threshold=0,   # 最低門檻
                display_mode='z',
                cut_coords=[-30, -10, 0, 10, 20, 30, 40],
                title=f"Fallback: Condition {cond}",
                colorbar=True,
                cmap='cold_hot',
                vmin=-13,
                vmax=13,
                symmetric_cbar=True
            )
            display.savefig(output_file)
            display.close()
        except Exception as e2:
            print(f"⚠️ 第二次也失敗: {e2}")

✅ 成功畫圖: Condition 1
✅ 成功畫圖: Condition 2
✅ 成功畫圖: Condition 3
✅ 成功畫圖: Condition 4
✅ 成功畫圖: Condition 5
✅ 成功畫圖: Condition 6
✅ 成功畫圖: Condition 7
✅ 成功畫圖: Condition 8
✅ 成功畫圖: Condition 9
✅ 成功畫圖: Condition 10
✅ 成功畫圖: Condition 11
✅ 成功畫圖: Condition 12
✅ 成功畫圖: Condition 13
✅ 成功畫圖: Condition 14
✅ 成功畫圖: Condition 15
✅ 成功畫圖: Condition 16
✅ 成功畫圖: Condition 17
✅ 成功畫圖: Condition 18
✅ 成功畫圖: Condition 19
✅ 成功畫圖: Condition 20
✅ 成功畫圖: Condition 21
✅ 成功畫圖: Condition 22
✅ 成功畫圖: Condition 23
✅ 成功畫圖: Condition 24
✅ 成功畫圖: Condition 25
✅ 成功畫圖: Condition 26
✅ 成功畫圖: Condition 27
✅ 成功畫圖: Condition 28
✅ 成功畫圖: Condition 29
✅ 成功畫圖: Condition 30
✅ 成功畫圖: Condition 31
✅ 成功畫圖: Condition 32
✅ 成功畫圖: Condition 33
✅ 成功畫圖: Condition 34
✅ 成功畫圖: Condition 35
✅ 成功畫圖: Condition 36
✅ 成功畫圖: Condition 37
✅ 成功畫圖: Condition 38
✅ 成功畫圖: Condition 39
✅ 成功畫圖: Condition 40
✅ 成功畫圖: Condition 41
✅ 成功畫圖: Condition 42
✅ 成功畫圖: Condition 43
✅ 成功畫圖: Condition 44
✅ 成功畫圖: Condition 45
✅ 成功畫圖: Condition 46
✅ 成功畫圖: Condition 47
✅ 成功畫圖: Condition 48
✅

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# 讀入檔案
fmri = pd.read_csv("wholebrain_mean_activation.csv")
behav = pd.read_csv("behavior.csv")  # 你準備的行為表現資料

# 合併資料（依 subject 與 condition 對應）
df = pd.merge(fmri, behav, on=["subject", "condition"])

# 顯示前幾列確認
print(df.head())

# 計算相關係數（全資料）
r_acc, p_acc = pearsonr(df["mean_beta"], df["accuracy"])
r_rt, p_rt = pearsonr(df["mean_beta"], df["rt"])

print(f"✅ beta vs accuracy: r = {r_acc:.3f}, p = {p_acc:.4f}")
print(f"✅ beta vs RT:       r = {r_rt:.3f}, p = {p_rt:.4f}")

# 畫圖：Beta vs Accuracy
sns.lmplot(data=df, x="mean_beta", y="accuracy", height=5, aspect=1.2)
plt.title(f"Correlation: beta vs accuracy (r={r_acc:.2f}, p={p_acc:.3g})")
plt.tight_layout()
plt.savefig("corr_beta_accuracy.png", dpi=300)

# 畫圖：Beta vs RT
sns.lmplot(data=df, x="mean_beta", y="rt", height=5, aspect=1.2)
plt.title(f"Correlation: beta vs RT (r={r_rt:.2f}, p={p_rt:.3g})")
plt.tight_layout()
plt.savefig("corr_beta_rt.png", dpi=300)

In [ ]:
#透明化
from PIL import Image

img = Image.open("cond_0001.png").convert("RGBA")
datas = img.getdata()

new_data = []
for item in datas:
    if item[:3] == (0, 0, 0):  # 黑色背景
        new_data.append((0, 0, 0, 0))  # 轉透明
    else:
        new_data.append(item)

img.putdata(new_data)
img.save("cond_0001_transparent.png", "PNG")